# Recommender Systems

Recommender systems are useful for recommending users items based on their past preferences. Recommender systems are generally classified into the following categories:

- **Content-based** based on similarity of item attributes (e.g. keywords, categories) and users preferences: For instance, if a user liked Item A, the recommender system would recommend items with similar characteristics. If we would talk about movies, those characteristics would include producer, genre, etc... Typically, this can be done with a binary classification supervised algorithm.


- **Collaborative filtering**, which calculates similarity from interactions (e.g. ratings, number of purchases, likes, etc.). This process finds customers who have similar preferences, and then gives recommendations to one customer from a set of items that are new to that particular customer and preferred by other customers with a similar profile. The system makes an assumption that users with similar movie watching habits have similar preferences in general. It finds users who watch videos similar to those the user watches, then identifies videos those similar users have watched, but which the first user has not. There are 3 approaches to this; user-user collaborative filtering, item-item collaborative filtering and matrix factorization.


- New approaches and usually more powerful are **Hybrid systems**, which combine the two aforementioned methods.

<img src="data/image.png" width=600>

The above diagram categorises in detail the different methodologies for building a recommender system.

In context to the `movieLens` that will be used later as a sample dataset:
- We will suggest different movies based on the content similarity, e.g. genre, cast, etc. applying an item-content filtering
- We could also compare the user metadata, such as age and gender, and suggest items to the user that similar users have liked. In that case, we would use a user-content filtering. The `movielens` dataset doesn't contain any user content data, so we will build only an item-item collaborative filtering.

**Memory-based** content filtering

In memory-based methods there is no model that learns from the data to predict. A pre-computed matrix of similarities is constructed that can be used for movie predic

__Load the datasets in three different pandas dataframes__

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load movies data
movies = pd.read_csv('data/movies.csv')
tags = pd.read_csv('data/tags.csv')
ratings = pd.read_csv('data/ratings.csv')

display(movies.head())
display(tags.head())
display(ratings.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/movies.csv'

__remove the '|' that separates the different genres of movies and replace with an empty space__

In [ ]:
movies['genres'] = movies['genres'].str.replace('|',' ')

__Filter the `ratings` dataframe, to have only movies that have received a rating__

In [ ]:
# Filter the movies df
movies = movies[movies.movieId.isin(ratings.movieId.unique().tolist())]
movies.shape

# Feature engineering

We're gonna create a new feature called `metadata`, that merges **all the text data information** we have about a movie : the genre and the tags

__Merge the `movies` and `tags` data frames__

In [ ]:
# Merge
merged = pd.merge(movies, tags, on='movieId', how='left')
# Drop useless columns.
merged.head()

__Create a new dataframe with the `metadata` column, containing a string of both tags AND genres merged together__

So you have to merge **tags first** together per movie. And **then** merge the combined tags with the **genres** column.

For example the `metadata` column of the movie Toy Story will be `pixar pixar fun Adventure Animation Children Comedy Fantasy`

In [ ]:
# Create a new df with a new metadata column from tags and genres.
merged.fillna("", inplace=True)
merged = pd.DataFrame(merged.groupby('movieId')['tag'].apply(lambda x: "%s" % ' '.join(x)))
df = pd.merge(movies, merged, on='movieId', how='left')
df['metadata'] = df[['tag', 'genres']].apply(lambda x: ' '.join(x), axis = 1)

df.head()

## Create a content latent matrix from metadata

### Count Vectorizer

Next we need to transform the metadata text to vectors, in order to feed it to our machine learning algorithms. Machine Learning models don't understand text data so we need to encode it.

We're gonna use the `CountVectorizer` to encode the metadata column. 

Create a new dataframe `count_df` with the vectors you get from this count transformation. Every line is gonna be the frequency vector for this specific movie.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['metadata'])
count_df = pd.DataFrame(count_matrix.toarray(), index=df.index.tolist())
count_df.head()

### Dimensionality reduction

Each movie metadata was transformed into a vector of length 1675! (approximately)

As we saw in previous lessons, we can apply dimensionality reduction methods to describe the data (movies) without much loss of information. Truncated singular value decomposition (SVD) is another advanced tool to reduce dimensions.

Contrary to PCA, this estimator **does not center the data before computing the singular value decomposition**. This means it can work with **sparse matrices efficiently**. In particular, truncated SVD works on term count/frequency matrices. In that context, it is known as latent semantic analysis (LSA).

You can look at `TruncatedSVD` class in scikit learn but it's the same principle as PCA

__👉apply the truncated SVD with a reduced number of features (say 25) to reduce the dimension of your Count matrix__

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Apply SVD
svd = TruncatedSVD(n_components=25)
latent_df = svd.fit_transform(count_df)

👉**Plot the cumulative explained variance ratio as a function of the number of components**

In [ ]:
# Plot variance as a function of the number of components.
plt.plot(svd.explained_variance_ratio_.cumsum())
plt.xlabel('number of singular value components')
plt.ylabel('Cumulative percent of variance')   
plt.grid()
plt.show()

We can see that with only the first 25 components (from initially about 1700) we can explain **more than 80%** of the variance, which suffices for our study purposes.

👉**save the 25 first components of this matrix into a new `latent_df` dataframe, indexed by the titles from the movies**

In [ ]:
# keep the first 25 number of latent components.
n = 25 
latent_df = pd.DataFrame(latent_df[:,0:n], index=df.title.tolist())
latent_df.shape

## Create a latent matrix from user ratings

Except from the metadata we have another valuable source of information, **the user ratings**. 

A recommender system can recommend a movie that is similar on the basis of user ratings (item-item collaborative filtering).

👉**We prepare the dataset below to have movies as lines, userids as columns**

In [ ]:
# Merge
ratings1 = pd.merge(movies[['movieId']], ratings, on="movieId", how="right")
# Pivot
ratings2 = ratings1.pivot(index = 'movieId', columns ='userId', values = 'rating').fillna(0)
display(ratings2.head())
ratings2.shape

We just created a dataset with movies as vectors of length 9724.

👉**Once again, we will apply SVD to the `ratings2` dataframe, so as to keep only the first 200 components. Call this dataframe `latent_df_2`**

In [ ]:
svd = TruncatedSVD(n_components=200)
latent_df_2 = svd.fit_transform(ratings2)
 
# Plot variance as a function of the number of components.
plt.plot(svd.explained_variance_ratio_.cumsum())
plt.xlabel('number of singular value components')
plt.ylabel('Cumulative percent of variance')   
plt.grid()
plt.show()

👉We reindex by the film title

In [ ]:
latent_df_2 = pd.DataFrame(latent_df_2, index=df.title.tolist())
latent_df_2.head()

## Apply cosine similarity in content and colaborative matrices

Next, we will use a similarity measure to find the top $N$ **most similar movies to "Toy Story"** based on the filtering methods we created. Cosine similarity is one of the similarity measures we can use (`sklearn.metrics.pairwise.cosine_similarity`)

__👉Compute the cosine similarity of a sample movie ("Toy Story") to both content and collaborative latent matrices.__

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Get the latent vectors for "Toy Story" from content and collaborative matrices
v1 = np.array(latent_df.loc['Toy Story (1995)']).reshape(1, -1)
v2 = np.array(latent_df_2.loc['Toy Story (1995)']).reshape(1, -1)

# Compute the cosine similartity of this movie with the others in the list
sim1 = cosine_similarity(latent_df, v1).reshape(-1)
sim2 = cosine_similarity(latent_df_2, v2).reshape(-1)

We can also create a hybrid filter, which is an average measure of similarity from both content and collaborative filtering. 

`hybrid-similarity = (content-similarity + collab-similarity) / 2`

__Calculate the average measure of both content and collaborative__

In [ ]:
hybrid = ((sim1 + sim2)/2.0)

__Create a dataframe with the final similarities to Toy Story__

In [ ]:
dictDf = {'content': sim1 , 'collaborative': sim2, 'hybrid': hybrid} 
recommendation_df = pd.DataFrame(dictDf, index = latent_df.index)

__Sort your dataframe by most similar by collaborative similarity__

In [ ]:
recommendation_df.sort_values('collaborative', ascending=False, inplace=True)
recommendation_df.head(10)

You should see Toy Story as the most similar movie of course (similarity of 1 for every column)

__You can sort on contant as well as hybrid and see which one gives the best recommentation__

In [ ]:
recommendation_df.sort_values('hybrid', ascending=False, inplace=True)
recommendation_df.head(10)